In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R4JEZKNKR to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: udacity-project
Azure region: eastus
Subscription id: 0c5a644d-c5ce-4e3b-bf42-4cb265317817
Resource group: aml-quickstarts-110433


In [33]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": choice(0.01, 0.1, 1),
    "max_iter": choice(100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

# script_params = {
#        '--C': 'linear',
#        '--penalty': 1.0,
#    }

est = SKLearn(source_directory="./",
                compute_target=cpu_cluster,
                entry_script='train.py'
            )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [16]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='sklearn-logistic', model_path='./')

best_run
## best run model of logistic regression has C = 0.1, max_iter = 100 and accuracy = 0.9109

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_e8d18854-5eb8-4fd3-a84f-04753c7eaee6_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

ds_train, ds_test = ds.random_split(percentage = 0.8, seed = 42)

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task= "classification",
    primary_metric="accuracy",
    training_data=ds_train,
    label_column_name="y",
    compute_target = cpu_cluster,
    n_cross_validations=5)

In [19]:
# Submit your automl run

automl_exp = Experiment(ws, "automl-experiment")

run = automl_exp.submit(automl_config, show_output=True)

Running on remote or ADB.
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_98640396-8d15-4beb-9499-5935d57c3563

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input dat

In [23]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_model, best_automl_run = run.get_output()
automl_model = best_automl_model.register_model(model_name='automl-model', model_path='./')

In [34]:
# Remove the AmlCompute object (cpu_cluster) from its associated workspace (udacity-project).
cpu_cluster.delete()

AmlCompute(workspace=Workspace.create(name='udacity-project', subscription_id='0c5a644d-c5ce-4e3b-bf42-4cb265317817', resource_group='aml-quickstarts-110433'), name=cpu-cluster, id=/subscriptions/0c5a644d-c5ce-4e3b-bf42-4cb265317817/resourceGroups/aml-quickstarts-110433/providers/Microsoft.MachineLearningServices/workspaces/udacity-project/computes/cpu-cluster, type=AmlCompute, provisioning_state=Creating, location=eastus, tags=None)

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

